## Create reports for several recordings

In [5]:
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')
import mne
from tinnsleep.config import Config
from tinnsleep.create_reports import preprocess, reporting
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
from tinnsleep.config import Config
print("config loaded")


config loaded


In [6]:
#Setting parameters
EDF_list = Config.bruxisme_files[0:1]
ind_picks_chan = [0, 4]
ind_picks_imp = [1, 5]
THR_classif=[[0,2], [0,3], [0,4], [0,5] , [0,6], [0,7], [0,8], [0,9], [0,10]]
sfreq = 200
window_length = 0.25                    # in seconds
duration = int(window_length * sfreq)   # in samples
interval = duration                     # no overlapping
THR_imp = 8000
params = dict(ch_names=['1', '2'],
                  rejection_thresholds=dict(emg=1e-04),  # two order of magnitude higher q0.01
                  flat_thresholds=dict(emg=1e-09),  # one order of magnitude lower median
                  channel_type_idx=dict(emg=[0, 1]),
                  full_report=True
                  )

print("parameters set")


parameters set


In [10]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)

    results={}
    for filename in EDF_list:
        raw = mne.io.read_raw_edf(filename, preload=False)  # prepare loading
        picks_chan=[]
        for elm in ind_picks_chan:
            picks_chan.append(raw.info["ch_names"][elm])
        picks_imp=[]
        for elm in ind_picks_imp:
            picks_imp.append(raw.info["ch_names"][elm])
        epochs, valid_labels, log = preprocess(raw, picks_chan, picks_imp, duration, interval, params, THR_imp=THR_imp, get_log=True)
        results[filename] = reporting(epochs, valid_labels, THR_classif, log)


Extracting EDF parameters from C:\Users\zeta\documents\EEG_polysomno\PSG2\1BA07_nuit_hab.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
burst list empty
burst list empty
burst list empty
burst list empty


In [14]:
print(results.keys())
plt.figure()
for elm in results.keys():
    nb_ep=[]
    for ep in results[elm]["reports"]:
        nb_ep.append(ep["Number of episodes per hour"])
    print(nb_ep)
    st = elm[-19:] + " len " + str(ep["Clean data duration"])
    plt.plot(nb_ep, label=st)
plt.title('Comparison for number of episodes per hour')
plt.legend()
print(results[Config.bruxisme_files[0]]["THR_classif"][4])
print(results[Config.bruxisme_files[0]]["reports"][4])

dict_keys(['C:\\Users\\zeta\\documents\\EEG_polysomno\\PSG2\\1BA07_nuit_hab.edf'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[21.08454329078325, 15.697303154812202, 7.895093302716196, 2.1363193642643825, 0.09288345062019054, 0.0, 0.0, 0.0, 0.0]
[0, 6]
{'Clean data duration': 38758.25, 'Total burst duration': 8.25, 'Total number of burst': 31, 'Number of bursts per hour': 2.879386969225907, 'Total number of episodes': 1, 'Number of bursts per episode': 31.0, 'Number of episodes per hour': 0.09288345062019054, 'Number of tonic episodes per hour': 0.0, 'Number of phasic episodes per hour': 0.023220862655047636, 'Number of mixed episodes per hour': 0.0, 'Mean duration of tonic episode': nan, 'Mean duration of phasic episode': 2.75, 'Mean duration of mixed episode': nan}


In [17]:
print(results.keys())
plt.figure()
for elm in results.keys():
    nb_ep=[]
    for ep in results[elm]["reports"]:
        nb_ep.append(ep["Total number of burst"])
    print(f"Number of bursts for each THR_classif for {elm[-19:]} : {nb_ep}")
    st = elm[-19:] + " len " + str(ep["Clean data duration"])
    plt.plot(nb_ep, label=st)
plt.title('Comparison for total number of bursts')
plt.legend()

dict_keys(['C:\\Users\\zeta\\documents\\EEG_polysomno\\PSG2\\1BA07_nuit_hab.edf'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Number of bursts for each THR_classif for \1BA07_nuit_hab.edf : [3192, 1654, 802, 283, 31, 0, 0, 0, 0]
